In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from transformers import ViTModel, AutoModel, AutoFeatureExtractor, AutoTokenizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
import aim
import torch.optim as optim

class CustomSigLIP(nn.Module):
    def __init__(self, vision_model_name, text_model_name, projection_dim=512):
        super(CustomSigLIP, self).__init__()

        self.vision_model = ViTModel.from_pretrained(vision_model_name)
        vision_config = self.vision_model.config
        self.image_projection = nn.Linear(vision_config.hidden_size, projection_dim)

        self.text_model = AutoModel.from_pretrained(text_model_name)

        text_config = self.text_model.config
        self.text_projection = nn.Linear(text_config.hidden_size, projection_dim)

        self.logit_scale = nn.Parameter(torch.ones([]) * torch.log(torch.tensor(1 / 0.07)))

    def forward(self, images, text):
        image_features = self.vision_model(pixel_values=images).pooler_output
        image_features = self.image_projection(image_features)
        image_features = nn.functional.normalize(image_features, dim=-1)

        text_features = self.text_model(**text).pooler_output
        text_features = self.text_projection(text_features)
        text_features = nn.functional.normalize(text_features, dim=-1)

        logit_scale = self.logit_scale.exp()
        logits = logit_scale * torch.matmul(image_features, text_features.t())
        return logits

    def encode_image(self, images):
        image_features = self.vision_model(pixel_values=images).pooler_output
        image_features = self.image_projection(image_features)
        return nn.functional.normalize(image_features, dim=-1)

    def encode_text(self, text):
        text_features = self.text_model(**text).pooler_output
        text_features = self.text_projection(text_features)
        return nn.functional.normalize(text_features, dim=-1)


# Helper function to load the custom model
def load_custom_siglip(vision_model_name="google/vit-base-patch16-224-in21k",
                       text_model_name="bert-base-uncased",
                       projection_dim=512):
    return CustomSigLIP(vision_model_name, text_model_name, projection_dim)


# Parameters
vision_model_name = "google/vit-base-patch16-224-in21k"
text_model_name = "bert-base-uncased"
projection_dim = 512
batch_size = 64
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model = load_custom_siglip(vision_model_name, text_model_name, projection_dim).to(device)

for param in model.vision_model.parameters():
    param.requires_grad = False

for param in model.text_model.parameters():
    param.requires_grad = False

for param in model.image_projection.parameters():
    param.requires_grad = True

for param in model.text_projection.parameters():
    param.requires_grad = True

model.logit_scale.requires_grad = True

print("Trainable parameters:")
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)

/data/kazanplova/anaconda3/envs/flux_train/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Trainable parameters:
logit_scale
image_projection.weight
image_projection.bias
text_projection.weight
text_projection.bias


In [2]:
!nvidia-smi

Sat Dec 14 12:36:38 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:08:01.0 Off |                    0 |
| N/A   33C    P0             73W /  400W |    1255MiB /  81920MiB |     19%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [3]:
# Data preparation
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224 (ViT input size)
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.Flowers102(root="./data", split="train", transform=train_transform, download=True)
val_dataset = datasets.Flowers102(root="./data", split="val", transform=val_transform, download=True)
test_dataset = datasets.Flowers102(root="./data", split="test", transform=val_transform, download=True)


train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=24)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=24)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=24)

# Load SigLIP model
model = CustomSigLIP(vision_model_name, text_model_name, projection_dim)
model.to(device)

label_names = [item.strip().strip("'").strip('"') for item in open('flower_labels.txt').read().split('\n')]
labels_to_texts = {idx: name for idx, name in enumerate(label_names)}


In [4]:
labels_to_texts

{0: 'pink primrose',
 1: 'hard-leaved pocket orchid',
 2: 'canterbury bells',
 3: 'sweet pea',
 4: 'english marigold',
 5: 'tiger lily',
 6: 'moon orchid',
 7: 'bird of paradise',
 8: 'monkshood',
 9: 'globe thistle',
 10: 'snapdragon',
 11: "colt's foot",
 12: 'king protea',
 13: 'spear thistle',
 14: 'yellow iris',
 15: 'globe-flower',
 16: 'purple coneflower',
 17: 'peruvian lily',
 18: 'balloon flower',
 19: 'giant white arum lily',
 20: 'fire lily',
 21: 'pincushion flower',
 22: 'fritillary',
 23: 'red ginger',
 24: 'grape hyacinth',
 25: 'corn poppy',
 26: 'prince of wales feathers',
 27: 'stemless gentian',
 28: 'artichoke',
 29: 'sweet william',
 30: 'carnation',
 31: 'garden phlox',
 32: 'love in the mist',
 33: 'mexican aster',
 34: 'alpine sea holly',
 35: 'ruby-lipped cattleya',
 36: 'cape flower',
 37: 'great masterwort',
 38: 'siam tulip',
 39: 'lenten rose',
 40: 'barbeton daisy',
 41: 'daffodil',
 42: 'sword lily',
 43: 'poinsettia',
 44: 'bolero deep blue',
 45: 'wall

In [5]:
def compute_metrics(predictions, targets):
    """
    Compute multi-class classification metrics.
    Args:
        predictions: Tensor of predicted class indices.
        targets: Tensor of ground truth class indices.
    Returns:
        dict: Metrics (accuracy, precision, recall, F1-score) including metrics per class.
    """

    predictions = predictions.cpu().numpy()
    targets = targets.cpu().numpy()

    # Общие метрики (по всем классам)
    metrics = {
        "accuracy": accuracy_score(targets, predictions),
        "precision_weighted": precision_score(targets, predictions, average="weighted", zero_division=0),
        "recall_weighted": recall_score(targets, predictions, average="weighted", zero_division=0),
        "f1_score_weighted": f1_score(targets, predictions, average="weighted", zero_division=0),
    }

    # Метрики по каждому классу
    per_class_precision = precision_score(targets, predictions, average=None, zero_division=0)
    per_class_recall = recall_score(targets, predictions, average=None, zero_division=0)
    per_class_f1 = f1_score(targets, predictions, average=None, zero_division=0)

    # Добавляем метрики по каждому классу в общий словарь
    for class_idx in range(len(per_class_precision)):
        metrics[f"precision_class_{class_idx}"] = per_class_precision[class_idx]
        metrics[f"recall_class_{class_idx}"] = per_class_recall[class_idx]
        metrics[f"f1_score_class_{class_idx}"] = per_class_f1[class_idx]

    return metrics

In [6]:


def train_with_logging(model, train_loader, labels_to_texts, tokenizer, optimizer, criterion, device, num_epochs, val_loader=None):
    model.train()

    for epoch in range(num_epochs):
        total_loss = 0.0
        all_predictions = []
        all_targets = []

        examples_to_log = []

        for images, labels in tqdm(train_loader):
            images = images.to(device)
            labels = labels.to(device)

            text_descriptions = [labels_to_texts[label.item()] for label in labels]
            encoded_texts = tokenizer(text_descriptions, return_tensors="pt", padding=True, truncation=True).to(device)

            logits = model(images, encoded_texts)

            targets = torch.arange(len(images)).to(device)
            loss = criterion(logits, targets)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            predictions = torch.argmax(logits, dim=1)
            all_predictions.append(predictions)
            all_targets.append(targets)

            if len(examples_to_log) < 5:  # Log the first 5 examples
                for img_idx in range(len(images)):
                    examples_to_log.append({
                        "image": images[img_idx].cpu(),
                        "true_label": labels[img_idx].item(),
                        "predicted_label": predictions[img_idx].item(),
                        "text_description": text_descriptions[img_idx]
                    })

        all_predictions = torch.cat(all_predictions, dim=0)
        all_targets = torch.cat(all_targets, dim=0)

        metrics = compute_metrics(all_predictions, all_targets)

        run.track(total_loss / len(train_loader), name='train_loss', step=epoch, context={"subset": "train"})
        for metric_name, metric_value in metrics.items():
            run.track(metric_value, name=metric_name, step=epoch, context={"subset": "train"})

        for example in examples_to_log:
            run.track(
                aim.Image(example["image"], caption=f"True: {example['true_label']}, Pred: {example['predicted_label']}, Text: {example['text_description']}"),
                name="classification_example",
                step=epoch,
                context={"subset": "train"}
            )

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(train_loader):.4f}")
        print(f"Train Metrics: {metrics}")
        print(f"Examples from epoch {epoch + 1}:")
        for example in examples_to_log:
            print({
                "true_label": example["true_label"],
                "predicted_label": example["predicted_label"],
                "text_description": example["text_description"]
            })

        if val_loader:
            val_metrics = test_with_logging(model, val_loader, labels_to_texts, tokenizer, device)
            print(f"Validation Metrics: {val_metrics}")
            

def test_with_logging(model, test_loader, labels_to_texts, tokenizer, device, log_prefix="test"):
    model.eval()
    all_predictions = []
    all_targets = []
    examples_to_log = []

    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images = images.to(device)
            labels = labels.to(device)

            text_descriptions = [labels_to_texts[label.item()] for label in labels]
            encoded_texts = tokenizer(text_descriptions, return_tensors="pt", padding=True, truncation=True).to(device)

            logits = model(images, encoded_texts)

            predictions = torch.argmax(logits, dim=1)
            all_predictions.append(predictions)
            all_targets.append(torch.arange(len(labels)).to(device))  # Target is the diagonal (index matches)

            if len(examples_to_log) < 5:  # Log the first 5 examples
                for img_idx in range(len(images)):
                    examples_to_log.append({
                        "image": images[img_idx].cpu(),
                        "true_label": labels[img_idx].item(),
                        "predicted_label": predictions[img_idx].item(),
                        "text_description": text_descriptions[img_idx]
                    })

    all_predictions = torch.cat(all_predictions, dim=0)
    all_targets = torch.cat(all_targets, dim=0)

    metrics = compute_metrics(all_predictions, all_targets)

    for metric_name, metric_value in metrics.items():
        run.track(metric_value, name=metric_name, context={"subset": log_prefix})

    for example in examples_to_log:
        run.track(
            aim.Image(example["image"], caption=f"True: {example['true_label']}, Pred: {example['predicted_label']}, Text: {example['text_description']}"),
            name="classification_example",
            context={"subset": log_prefix}
        )

    print(f"Examples from {log_prefix} set:")
    for example in examples_to_log:
        print({
            "true_label": example["true_label"],
            "predicted_label": example["predicted_label"],
            "text_description": example["text_description"]
        })

    return metrics

In [7]:
num_epochs = 50
learning_rate = 1e-5

run = aim.Run(repo='.', experiment='multi_class_classification_flowers102')
run["hparams"] = {
    "vision_model_name": vision_model_name,
    "text_model_name": text_model_name,
    "projection_dim": projection_dim,
    "batch_size": batch_size,
    "num_epochs": num_epochs,
    "learning_rate": learning_rate,
}

In [ ]:
feature_extractor = AutoFeatureExtractor.from_pretrained(vision_model_name)
tokenizer = AutoTokenizer.from_pretrained(text_model_name)

optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)
criterion = nn.CrossEntropyLoss()

train_with_logging(model, train_loader, labels_to_texts, tokenizer, optimizer, criterion, device, num_epochs, val_loader)

test_metrics = test_with_logging(model, test_loader, labels_to_texts, tokenizer, device)
print(f"Test Metrics: {test_metrics}")

run.close()

/data/kazanplova/anaconda3/envs/flux_train/lib/python3.11/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
/data/kazanplova/anaconda3/envs/flux_train/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
100%|██████████| 16/16 [00:08<00:00,  1.91it/s]


Epoch [1/50], Loss: 4.0933
Train Metrics: {'accuracy': 0.043137254901960784, 'precision_weighted': 0.04155930739331908, 'recall_weighted': 0.043137254901960784, 'f1_score_weighted': 0.04024732269588744, 'precision_class_0': 0.0, 'recall_class_0': 0.0, 'f1_score_class_0': 0.0, 'precision_class_1': 0.0, 'recall_class_1': 0.0, 'f1_score_class_1': 0.0, 'precision_class_2': 0.0, 'recall_class_2': 0.0, 'f1_score_class_2': 0.0, 'precision_class_3': 0.058823529411764705, 'recall_class_3': 0.0625, 'f1_score_class_3': 0.06060606060606061, 'precision_class_4': 0.02702702702702703, 'recall_class_4': 0.0625, 'f1_score_class_4': 0.03773584905660377, 'precision_class_5': 0.09090909090909091, 'recall_class_5': 0.0625, 'f1_score_class_5': 0.07407407407407407, 'precision_class_6': 0.0, 'recall_class_6': 0.0, 'f1_score_class_6': 0.0, 'precision_class_7': 0.0, 'recall_class_7': 0.0, 'f1_score_class_7': 0.0, 'precision_class_8': 0.0, 'recall_class_8': 0.0, 'f1_score_class_8': 0.0, 'precision_class_9': 0.0,

  0%|          | 0/16 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true

Examples from test set:
{'true_label': 0, 'predicted_label': 0, 'text_description': 'pink primrose'}
{'true_label': 0, 'predicted_label': 20, 'text_description': 'pink primrose'}
{'true_label': 0, 'predicted_label': 30, 'text_description': 'pink primrose'}
{'true_label': 0, 'predicted_label': 10, 'text_description': 'pink primrose'}
{'true_label': 0, 'predicted_label': 30, 'text_description': 'pink primrose'}
{'true_label': 0, 'predicted_label': 40, 'text_description': 'pink primrose'}
{'true_label': 0, 'predicted_label': 40, 'text_description': 'pink primrose'}
{'true_label': 0, 'predicted_label': 30, 'text_description': 'pink primrose'}
{'true_label': 0, 'predicted_label': 30, 'text_description': 'pink primrose'}
{'true_label': 0, 'predicted_label': 20, 'text_description': 'pink primrose'}
{'true_label': 1, 'predicted_label': 30, 'text_description': 'hard-leaved pocket orchid'}
{'true_label': 1, 'predicted_label': 30, 'text_description': 'hard-leaved pocket orchid'}
{'true_label': 1,

  0%|          | 0/16 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true

...

  0%|          | 0/16 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true

Epoch [17/50], Loss: 0.3707
Train Metrics: {'accuracy': 0.7725490196078432, 'precision_weighted': 0.8119546263905334, 'recall_weighted': 0.7725490196078432, 'f1_score_weighted': 0.7720463756099826, 'precision_class_0': 0.5925925925925926, 'recall_class_0': 1.0, 'f1_score_class_0': 0.7441860465116279, 'precision_class_1': 0.6666666666666666, 'recall_class_1': 1.0, 'f1_score_class_1': 0.8, 'precision_class_2': 0.5357142857142857, 'recall_class_2': 0.9375, 'f1_score_class_2': 0.6818181818181818, 'precision_class_3': 0.64, 'recall_class_3': 1.0, 'f1_score_class_3': 0.7804878048780488, 'precision_class_4': 0.5925925925925926, 'recall_class_4': 1.0, 'f1_score_class_4': 0.7441860465116279, 'precision_class_5': 0.6363636363636364, 'recall_class_5': 0.875, 'f1_score_class_5': 0.7368421052631579, 'precision_class_6': 0.7368421052631579, 'recall_class_6': 0.875, 'f1_score_class_6': 0.8, 'precision_class_7': 0.7647058823529411, 'recall_class_7': 0.8125, 'f1_score_class_7': 0.7878787878787878, 'pre

  0%|          | 0/16 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true

Examples from test set:
{'true_label': 0, 'predicted_label': 0, 'text_description': 'pink primrose'}
{'true_label': 0, 'predicted_label': 0, 'text_description': 'pink primrose'}
{'true_label': 0, 'predicted_label': 0, 'text_description': 'pink primrose'}
{'true_label': 0, 'predicted_label': 0, 'text_description': 'pink primrose'}
{'true_label': 0, 'predicted_label': 0, 'text_description': 'pink primrose'}
{'true_label': 0, 'predicted_label': 0, 'text_description': 'pink primrose'}
{'true_label': 0, 'predicted_label': 0, 'text_description': 'pink primrose'}
{'true_label': 0, 'predicted_label': 0, 'text_description': 'pink primrose'}
{'true_label': 0, 'predicted_label': 0, 'text_description': 'pink primrose'}
{'true_label': 0, 'predicted_label': 0, 'text_description': 'pink primrose'}
{'true_label': 1, 'predicted_label': 10, 'text_description': 'hard-leaved pocket orchid'}
{'true_label': 1, 'predicted_label': 10, 'text_description': 'hard-leaved pocket orchid'}
{'true_label': 1, 'predict

  0%|          | 0/16 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true